# **Ensemble Methods - Feature Sert 3**

We have now trained 6 models for each of our sets of features. These models show some performance but still lack the ideal performance required in order for them to be deployed in clinical environments.

The purpose of creating ensemble predictions is to determine whether combining the outputs of multiple base models can achieve better performance than the individual models

As a final experiment, we will look at implementing a few ensemble methods to see if a combination of each of the models has better predictive performance than any of the models individually.

The potential benefits of this are:
- Reduction in overfitting
- More robust
- Combine diverse model strengths (LSTM, TCN look at time dependencies where as LGBM looks at aggregate values over time)
- Models handle different types of data (static and dynamic)
- Enhanced generalisation
- Mitigation of individual model limitations

We will need to create ensemble methods for each feature set.

We will investigate different revelant ensemble methods:
- Averaging
- Majority voting
- Stacking
- Potentially more complex methods e.g. Boosting, Bootstrapping

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Step 1 - Collate predictions from each model**

In [ ]:
# Load model predictions
lstm_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/01_lstm_fused_decision/dynamic_data/best_models/experiment_1/predictions.npy')
lstm_static_and_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/01_lstm_fused_decision/static_and_dynamic_data/best_models/predictions.npy')
tcn_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/02_tcn_fused_decision/dynamic_data/best_models/experiment_1/predictions.npy')
tcn_static_and_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/02_tcn_fused_decision/static_and_dynamic_data/best_models/predictions.npy')
lgbm_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/03_lgbm_model/dynamic_data/best_models/predictions.npy')
lgbm_static_and_dynamic_predictions = np.load('/content/drive/MyDrive/MSc_Final_Project/03_model_development/03_feature_set_3/03_lgbm_model/static_and_dynamic_data/best_models/predictions_v2.npy')

# Print the shapes of all predictions
print(f"LSTM Dynamic Predictions Shape: {lstm_dynamic_predictions.shape}")
print(f"LSTM Static and Dynamic Predictions Shape: {lstm_static_and_dynamic_predictions.shape}")
print(f"TCN Dynamic Predictions Shape: {tcn_dynamic_predictions.shape}")
print(f"TCN Static and Dynamic Predictions Shape: {tcn_static_and_dynamic_predictions.shape}")
print(f"LGBM Dynamic Predictions Shape: {lgbm_dynamic_predictions.shape}")
print(f"LGBM Static and Dynamic Predictions Shape: {lgbm_static_and_dynamic_predictions.shape}")

LSTM Dynamic Predictions Shape: (941,)
LSTM Static and Dynamic Predictions Shape: (941,)
TCN Dynamic Predictions Shape: (941,)
TCN Static and Dynamic Predictions Shape: (941,)
LGBM Dynamic Predictions Shape: (941,)
LGBM Static and Dynamic Predictions Shape: (941,)


In [ ]:
# Print a sample of each
print("Sample of LSTM Dynamic Predictions:")
print(lstm_dynamic_predictions[:5])
print("\nSample of LSTM Static and Dynamic Predictions:")
print(lstm_static_and_dynamic_predictions[:5])
print("\nSample of TCN Dynamic Predictions:")
print(tcn_dynamic_predictions[:5])
print("\nSample of TCN Static and Dynamic Predictions:")
print(tcn_static_and_dynamic_predictions[:5])
print("\nSample of LGBM Dynamic Predictions:")
print(lgbm_dynamic_predictions[:5])
print("\nSample of LGBM Static and Dynamic Predictions:")
print(lgbm_static_and_dynamic_predictions[:5])

Sample of LSTM Dynamic Predictions:
[0.48093346 0.64630634 0.37296134 0.54641134 0.4082523 ]

Sample of LSTM Static and Dynamic Predictions:
[0.486189   0.48342422 0.32294074 0.32915205 0.37674662]

Sample of TCN Dynamic Predictions:
[0.26872873 0.37689948 0.20276345 0.25100258 0.24051048]

Sample of TCN Static and Dynamic Predictions:
[0.5103498  0.68355703 0.3994471  0.55919015 0.3422651 ]

Sample of LGBM Dynamic Predictions:
[0.85215587 0.39560889 0.32495838 0.57319537 0.77139334]

Sample of LGBM Static and Dynamic Predictions:
[0.73190767 0.33879724 0.36417136 0.37155295 0.800932  ]


In [ ]:
# Combine all predictions into a single array
all_predictions = np.vstack([
    lstm_dynamic_predictions,
    lstm_static_and_dynamic_predictions,
    tcn_dynamic_predictions,
    tcn_static_and_dynamic_predictions,
    lgbm_dynamic_predictions,
    lgbm_static_and_dynamic_predictions
])

all_predictions.shape

(6, 941)

In [ ]:
# Load the true labels for the test set
test_labels_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/08_model_input_data/03_feature_set_3/01_lstm_data/dynamic_data_v2/test_labels_v1.npy'
test_labels = np.load(test_labels_path)

# Print the shape of the test labels
print(f"Test Labels Shape: {test_labels.shape}")

Test Labels Shape: (941,)


In [ ]:
test_labels.astype(int)

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,

# **Step 2 - Apply Ensemble methods**

### **Average Predictions**

Average the probabilities for each patient and evaluate on the test labels.

In [ ]:
# Compute the average prediction for each sample
ensemble_predictions = np.mean(all_predictions, axis=0)

In [ ]:
ensemble_predictions_df = pd.DataFrame(ensemble_predictions)

ensemble_predictions_df.to_parquet('/content/drive/MyDrive/MSc_Final_Project/04_ensemble_methods/results/fs3_avg.parquet')

In [ ]:
# Define standard binary classification threshold of 0.5
threshold = 0.5

ensemble_predictions_binary = (ensemble_predictions >= threshold).astype(int)

In [ ]:
ensemble_predictions_binary.shape

(941,)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [ ]:
# Evaluate ensemble predictions
def evaluate_ensemble_predictions(ensemble_predictions, test_labels):
    # Ensure predictions and labels are binary and of the same type
    ensemble_predictions = np.array(ensemble_predictions).astype(int)
    test_labels = np.array(test_labels).astype(int)

    # Evaluate the ensemble prediction against the test labels
    accuracy = accuracy_score(test_labels, ensemble_predictions)
    precision = precision_score(test_labels, ensemble_predictions)
    recall = recall_score(test_labels, ensemble_predictions)
    f1 = f1_score(test_labels, ensemble_predictions)
    roc_auc = roc_auc_score(test_labels, ensemble_predictions)

    # Compute confusion matrix to derive specificity
    tn, fp, fn, tp = confusion_matrix(test_labels, ensemble_predictions).ravel()
    specificity = tn / (tn + fp)

    # Print the metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall (Sensitivity): {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Specificity: {specificity:.4f}')

In [ ]:
# Evaluate averaging prediction
evaluate_ensemble_predictions(ensemble_predictions_binary, test_labels)

Accuracy: 0.6706
Precision: 0.4962
Recall (Sensitivity): 0.2104
F1 Score: 0.2955
ROC AUC: 0.5530
Specificity: 0.8956


### **Hard voting classifier**

Each model hard votes for a class.

In [ ]:
# Ensure all predictions are binary
lstm_dynamic_predictions_binary = (lstm_dynamic_predictions >= threshold).astype(int)
lstm_static_and_dynamic_predictions_binary = (lstm_static_and_dynamic_predictions >= threshold).astype(int)
tcn_dynamic_predictions_binary = (tcn_dynamic_predictions >= threshold).astype(int)
tcn_static_and_dynamic_predictions_binary = (tcn_static_and_dynamic_predictions >= threshold).astype(int)
lgbm_dynamic_predictions_binary = (lgbm_dynamic_predictions >= threshold).astype(int)
lgbm_static_and_dynamic_predictions_binary = (lgbm_static_and_dynamic_predictions >= threshold).astype(int)

In [ ]:
all_predictions_binary = np.vstack([
    lstm_dynamic_predictions_binary,
    lstm_static_and_dynamic_predictions_binary,
    tcn_dynamic_predictions_binary,
    tcn_static_and_dynamic_predictions_binary,
    lgbm_dynamic_predictions_binary,
    lgbm_static_and_dynamic_predictions_binary
]).astype(int)

In [ ]:
# Apply hard voting
ensemble_predictions_hard = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=all_predictions_binary)

# Evaluate the ensemble predictions
evaluate_ensemble_predictions(ensemble_predictions_hard, test_labels)

Accuracy: 0.6706
Precision: 0.4945
Recall (Sensitivity): 0.1456
F1 Score: 0.2250
ROC AUC: 0.5364
Specificity: 0.9272


### **Stacking**


Stacking uses a meta-model to learn how to best combine the predictions from two or more base mdoels. This allows it to harness the capabilities of a range of models.



Train Base Models: Multiple base models are trained on the training data. These models can be of different types (e.g., logistic regression, random forest, SVM). **- This has already been done with LSTM, TCN and LGBM**

Generate Meta-Features: The predictions from the base models are used to create a new dataset. Each base model's prediction for each sample becomes a feature in this new dataset. This dataset is called the meta-features.

Train Meta-Model: A meta-model (or level-2 model) is trained on the meta-features. The goal of the meta-model is to learn how to combine the base model predictions to make the final prediction.

Make Final Predictions: The meta-model uses the meta-features (predictions from the base models) to make the final predictions.

For this we will use the standard simple meta-model: Logistic Regression as linear models are often used as the meta-model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Combine predictions into a feature matrix for the meta-model
X_meta = np.vstack([
    lstm_dynamic_predictions,
    lstm_static_and_dynamic_predictions,
    tcn_dynamic_predictions,
    tcn_static_and_dynamic_predictions,
    lgbm_dynamic_predictions,
    lgbm_dynamic_predictions
]).T

In [ ]:
# Split meta features into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_meta, test_labels, test_size=0.2, random_state=42)

In [ ]:
# Define the meta-model
meta_model = LogisticRegression()

In [ ]:
# Train the meta-model
meta_model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
ensemble_predictions_stacking_prob = meta_model.predict_proba(X_val)[:, 1]

ensemble_predictions_stacking_prob[:5]

array([0.27507808, 0.46695122, 0.34460133, 0.37470979, 0.32895304])

In [ ]:
ensemble_predictions_stacking_prob_df = pd.DataFrame(ensemble_predictions_stacking_prob)

ensemble_predictions_stacking_prob_df.to_parquet('/content/drive/MyDrive/MSc_Final_Project/04_ensemble_methods/results/fs3_stacking.parquet')

In [ ]:
# Predict using the validation data
ensemble_predictions_stacking = meta_model.predict(X_val)

In [ ]:
# Evaluate on the test labels
evaluate_ensemble_predictions(ensemble_predictions_stacking, y_val)

Accuracy: 0.7249
Precision: 0.7200
Recall (Sensitivity): 0.2857
F1 Score: 0.4091
ROC AUC: 0.6151
Specificity: 0.9444


### **AdaBoost**

Boosting framework to improve the performance of the ensemble model.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)

In [ ]:
# Train the AdaBoost model
adaboost_model.fit(X_train, y_train)

# Predict
ensemble_predictions_adaboost = adaboost_model.predict(X_val)

In [ ]:
# Evaluate ensemble predictions
evaluate_ensemble_predictions(ensemble_predictions_adaboost, y_val)

Accuracy: 0.6825
Precision: 0.5385
Recall (Sensitivity): 0.3333
F1 Score: 0.4118
ROC AUC: 0.5952
Specificity: 0.8571


### **Bagging**

Reduces variance and helps avoid overfitting by averaging predictions over multiple models trained on different subsets.

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bagging_model = BaggingClassifier(n_estimators=50, random_state=42)

In [ ]:
# Train and predict
bagging_model.fit(X_train, y_train)
ensemble_predictions_bagging = bagging_model.predict(X_val)

In [ ]:
# Evaluate
evaluate_ensemble_predictions(ensemble_predictions_bagging, y_val)

Accuracy: 0.6402
Precision: 0.4528
Recall (Sensitivity): 0.3810
F1 Score: 0.4138
ROC AUC: 0.5754
Specificity: 0.7698


### **Optimal weights for each model**

We can tune what weights applied to the predictions for each model provides the best performance on ROC AUC basis.

In [ ]:
from scipy.optimize import differential_evolution
from sklearn.metrics import roc_auc_score

In [ ]:
# Define the objective function to minimize (negative ROC AUC)
def objective(weights):
    # Ensure weights sum to 1
    weights = np.array(weights)
    if np.sum(weights) != 1:
        weights = weights / np.sum(weights)

    # Calculate the weighted average of predictions
    weighted_predictions = np.average(all_predictions, axis=0, weights=weights)

    # Calculate the negative ROC AUC
    return -roc_auc_score(test_labels, weighted_predictions)

# Bounds: Each weight is between 0 and 1
bounds = [(0, 1)] * all_predictions.shape[0]

# Initial guess (equal weights)
initial_guess = [1 / all_predictions.shape[0]] * all_predictions.shape[0]

# Optimize weights using differential evolution
result = differential_evolution(objective, bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=1e-6, mutation=(0.5, 1), recombination=0.7, seed=42)

# Get the best weights
best_weights = result.x

# Calculate the final weighted predictions using the best weights
final_weighted_predictions = np.average(all_predictions, axis=0, weights=best_weights)

In [ ]:
# Evaluate the final weighted predictions
final_roc_auc = roc_auc_score(test_labels, final_weighted_predictions)

In [ ]:
# Convert weighted predictions to binary labels using a threshold of 0.5
final_binary_predictions = (final_weighted_predictions > 0.5).astype(int)

final_accuracy = accuracy_score(test_labels, final_binary_predictions)
final_precision = precision_score(test_labels, final_binary_predictions)
final_recall = recall_score(test_labels, final_binary_predictions)
final_f1 = f1_score(test_labels, final_binary_predictions)
tn, fp, fn, tp = confusion_matrix(test_labels, final_binary_predictions).ravel()
final_specificity = tn / (tn + fp)

# Print all evaluation metrics
print(f'Optimal Weights: {best_weights}')
print(f'Final ROC AUC: {final_roc_auc:.4f}')
print(f'Final Accuracy: {final_accuracy:.4f}')
print(f'Final Precision: {final_precision:.4f}')
print(f'Final Recall (Sensitivity): {final_recall:.4f}')
print(f'Final F1 Score: {final_f1:.4f}')
print(f'Final Specificity: {final_specificity:.4f}')

Optimal Weights: [0.01361065 0.89358719 0.6033816  0.26391157 0.00360392 0.00167867]
Final ROC AUC: 0.6705
Final Accuracy: 0.7109
Final Precision: 0.6164
Final Recall (Sensitivity): 0.3172
Final F1 Score: 0.4188
Final Specificity: 0.9035
